In [1]:
import pandas as pd
import os
import sys
sys.path.insert(0, os.path.abspath('../lib'))
from c19us_nyt import df_us as df
import numpy as np

In [2]:
states = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2011_us_ag_exports.csv')\
    [['code','state']]
states.columns = ['Code', 'State']
states.head()

,Code,State
0,AL,Alabama
1,AK,Alaska
2,AZ,Arizona
3,AR,Arkansas
4,CA,California


In [3]:
df = df.reset_index()[['date','fips','day','county','state','cases','deaths']]
df.columns = ['Date','FIPS','Day','County','State','Cases','Deaths']
print(df.shape)
df.head()

(72056, 7)


,Date,FIPS,Day,County,State,Cases,Deaths
0,2020-01-21,53061,0,Snohomish,Washington,1,0
1,2020-01-22,53061,1,Snohomish,Washington,1,0
2,2020-01-23,53061,2,Snohomish,Washington,1,0
3,2020-01-24,17031,3,Cook,Illinois,1,0
4,2020-01-24,53061,3,Snohomish,Washington,1,0


In [4]:
df_ = pd.merge(df, states, how='left', on='State')

In [5]:
df_.head()

,Date,FIPS,Day,County,State,Cases,Deaths,Code
0,2020-01-21,53061,0,Snohomish,Washington,1,0,WA
1,2020-01-22,53061,1,Snohomish,Washington,1,0,WA
2,2020-01-23,53061,2,Snohomish,Washington,1,0,WA
3,2020-01-24,17031,3,Cook,Illinois,1,0,IL
4,2020-01-24,53061,3,Snohomish,Washington,1,0,WA


In [6]:
all_counties = pd.read_csv('../../general_data/county_fips.csv')
all_counties['County'] = all_counties.County.map(lambda x: x.replace('County','').strip())
all_counties.head()

,FIPS,County,Code
0,0,UNITED STATES,NaN
1,1000,ALABAMA,NaN
2,1001,Autauga,AL
3,1003,Baldwin,AL
4,1005,Barbour,AL


In [7]:
for d in set(df_['Date']):
    if os.path.exists(f'../../county_data/date_{str(d).split(" ")[0]}.csv'):
        print('Already have file: ' + f'../../covid-19/data/county_data/date_{str(d).split(" ")[0]}.csv')
        pass
    else:
        print('Testing... ' + f'../../covid-19/data/county_data/date_{str(d).split(" ")[0]}.csv')
        dff = df_[df_['Date'] == d]
        added_counties = pd.merge(dff, all_counties, how='right', on='FIPS')
        added_counties = added_counties.drop(['County_x','Code_x'], axis=1)
        added_counties.columns = ['Date','FIPS','Day','State','Cases','Deaths','County','Code']
        added_counties['Cases'] = added_counties['Cases'].map(lambda x: 0 if np.isnan(x) else int(x))
        added_counties['Deaths'] = added_counties['Deaths'].map(lambda x: 0 if np.isnan(x) else int(x))
        added_counties['FIPS'] = added_counties['FIPS'].map(lambda x: '0' + str(x) if (int(x) <= 9999) else str(x))

        added_counties.to_csv(f'../../covid-19/data/county_data/date_{str(d).split(" ")[0]}.csv', index=False)

Testing... ../../covid-19/data/county_data/date_2020-04-04.csv
Testing... ../../covid-19/data/county_data/date_2020-01-27.csv
Testing... ../../covid-19/data/county_data/date_2020-01-26.csv
Testing... ../../covid-19/data/county_data/date_2020-04-15.csv
Testing... ../../covid-19/data/county_data/date_2020-03-29.csv
Testing... ../../covid-19/data/county_data/date_2020-03-03.csv
Testing... ../../covid-19/data/county_data/date_2020-02-22.csv
Testing... ../../covid-19/data/county_data/date_2020-02-09.csv
Testing... ../../covid-19/data/county_data/date_2020-03-17.csv
Testing... ../../covid-19/data/county_data/date_2020-02-24.csv
Testing... ../../covid-19/data/county_data/date_2020-03-27.csv
Testing... ../../covid-19/data/county_data/date_2020-02-01.csv
Testing... ../../covid-19/data/county_data/date_2020-01-30.csv
Testing... ../../covid-19/data/county_data/date_2020-02-04.csv
Testing... ../../covid-19/data/county_data/date_2020-02-23.csv
Testing... ../../covid-19/data/county_data/date_2020-04

In [8]:
df2 = df[['Date','State','Day','Cases','Deaths']]\
    .groupby(['Date','State','Day'])\
    .sum()\
    .reset_index()

In [9]:
df3 = pd.merge(df2, states, how='left', on='State')

In [10]:
print(df3.shape)
df3.head()

(2493, 6)


,Date,State,Day,Cases,Deaths,Code
0,2020-01-21,Washington,0,1,0,WA
1,2020-01-22,Washington,1,1,0,WA
2,2020-01-23,Washington,2,1,0,WA
3,2020-01-24,Illinois,3,1,0,IL
4,2020-01-24,Washington,3,1,0,WA


In [11]:
df3.to_csv('../../covid-19/data/daily_cases_USA_states.csv', index=False)

In [12]:
print(pd.read_csv('../../covid-19/data/daily_cases_USA_states.csv').shape)
pd.read_csv('../../covid-19/data/daily_cases_USA_states.csv').head()

(2493, 6)


,Date,State,Day,Cases,Deaths,Code
0,2020-01-21,Washington,0,1,0,WA
1,2020-01-22,Washington,1,1,0,WA
2,2020-01-23,Washington,2,1,0,WA
3,2020-01-24,Illinois,3,1,0,IL
4,2020-01-24,Washington,3,1,0,WA


In [13]:
df4 = df_[['Date', 'County','Cases','Deaths','Code']]\
    .groupby(['Date','County','Code'])\
    .sum()\
    .reset_index()

In [14]:
print(df4.shape)
df4.head()

(72012, 5)


,Date,County,Code,Cases,Deaths
0,2020-01-21,Snohomish,WA,1,0
1,2020-01-22,Snohomish,WA,1,0
2,2020-01-23,Snohomish,WA,1,0
3,2020-01-24,Cook,IL,1,0
4,2020-01-24,Snohomish,WA,1,0


In [15]:
df4.to_csv('../../covid-19/data/daily_cases_USA_counties.csv', index=False)

In [16]:
print(pd.read_csv('../../covid-19/data/daily_cases_USA_counties.csv').shape)
pd.read_csv('../../covid-19/data/daily_cases_USA_counties.csv').head()

(72012, 5)


,Date,County,Code,Cases,Deaths
0,2020-01-21,Snohomish,WA,1,0
1,2020-01-22,Snohomish,WA,1,0
2,2020-01-23,Snohomish,WA,1,0
3,2020-01-24,Cook,IL,1,0
4,2020-01-24,Snohomish,WA,1,0


In [17]:
df4.Date.min(), df4.Date.max()

(Timestamp('2020-01-21 00:00:00'), Timestamp('2020-04-19 00:00:00'))